## <span style="color:orange">Función Encapsulada</span>

**La función desarrollada en el punto 6 tiene como objetivo encapsular todos los pasos necesarios para transformar y limpiar los datasets originales, de manera que estén listos para ser utilizados en análisis posteriores o modelado predictivo.** 

**Este enfoque permite:**

- **Reutilización:** Al separar la lógica de limpieza en un módulo independiente, podemos importarla y aplicar las transformaciones sin duplicar lo ya realizado.
- **Estandarización:** Garantizamos que todos los datasets procesados sigan las mismas reglas de limpieza y transformación, lo que elimina inconsistencias y facilita la comparación entre diferentes datasets.
- **Optimización del Tiempo:** Al trabajar con datasets ya procesados, evitamos realizar tareas de limpieza repetitivas, mejorando la eficiencia en el desarrollo del proyecto.
  
**¿Por Qué Presentarla como Módulo?**

- Este diseño modular es una práctica común en proyectos de análisis y desarrollo de modelos, ya que:

    - **Aumenta la claridad del código:** Separar las transformaciones en un archivo dedicado ayuda a mantener el código principal más limpio y fácil de entender.
    - **Facilita la colaboración:** Otros miembros del equipo pueden usar o mejorar la función sin afectar el resto del proyecto.
    - **Evita errores:** Trabajar directamente con datasets ya limpios reduce la probabilidad de cometer errores en transformaciones repetidas.

---

**<span style="color:olive">Descripción de la Función</span>**

La **función procesar_dataset** realiza las siguientes operaciones clave:

1. **Creación de Variables Derivadas:**

    - Variables basadas en fechas, como is_weekend, is_holiday, y is_high_season, que capturan patrones estacionales y de comportamiento.
    - Conversión de stop a una métrica numérica (stop_numeric).
    - Cálculo de distancias entre ciudades (distance).

2. **Validación de Datos:**

    - Elimina columnas redundantes, como time_taken si ya existe time_in_minutes.
    - Marca valores atípicos (is_outlier), útiles para análisis exploratorios y modelado.

3. **Codificación de Variables Categóricas:** Transforma columnas como day_of_week en variables numéricas mediante codificación one-hot.

4. **Preparación para el Modelado:** Devuelve un dataset listo para ser utilizado en modelos predictivos o análisis adicionales.

---

**Importación en el Proyecto**

Al colocar la función en un archivo separado,**Función Encapsulada Punto 6_Prueba.ipynb**, podemos integrarla fácilmente en otros scripts o notebooks del proyecto. 

**<span style="color:green">import import_ipynb</span>**

**<span style="color:green">from Funcion_Encapsulada_Punto_6_Prueba import procesar_dataset</span>**

---

- El motivo de presentar esta función como un módulo independiente es demostrar una práctica profesional en el manejo de proyectos de análisis de datos. 

- Esto incluye:

    - **Modularidad:** Dividir el proyecto en componentes manejables y reutilizables.
    - **Escalabilidad:** Facilitar la extensión del proyecto en el futuro, permitiendo agregar nuevas transformaciones sin afectar el flujo existente.
    - **Mantenimiento:** Mejorar la capacidad de depuración y actualización del código.

**Esta estrategia refleja un enfoque ordenado, eficiente y profesional en el desarrollo de proyectos de análisis y modelado de datos, alineado con estándares de la industria.**

In [ ]:
def procesar_dataset(df, dataset_name):
    """
    Procesa un dataset original y devuelve un dataset limpio y procesado.
    
    Incluye:
    - Transformaciones y variables adicionales (basadas en el punto 5).
    - Limpieza de datos.
    - Validación de redundancias y eliminación de columnas innecesarias.
    - Codificación de variables categóricas.

    Parámetros:
        df (DataFrame): Dataset original a procesar.
        dataset_name (str): Nombre del dataset (por ejemplo, 'Business' o 'Economy').

    Retorna:
        DataFrame limpio y procesado.
    """
    # Crear una copia del DataFrame original
    df = df.copy()

    # ----------------------
    # Identificar el dataset
    # ----------------------
    df['dataset_name'] = dataset_name

    # ----------------------
    # 1. Variables basadas en 'date'
    # ----------------------
    df['date'] = pd.to_datetime(df['date'])
    df['is_weekend'] = df['date'].dt.dayofweek >= 5
    holidays = pd.to_datetime(["2022-01-01", "2022-12-25", "2022-07-04", "2022-11-24"])
    df['is_holiday'] = df['date'].isin(holidays)
    high_season_months = [12, 1, 7]
    df['is_high_season'] = df['date'].dt.month.isin(high_season_months)
    df['day_of_week'] = df['date'].dt.day_name()

    # ----------------------
    # 2. Incorporar distancia entre ciudades
    # ----------------------
    distances = {
        ("Delhi", "Mumbai"): 1400,
        ("Delhi", "Bangalore"): 2100,
    }
    df['distance'] = df.apply(
        lambda row: distances.get((row['from'], row['to']), np.nan), axis=1
    )

    # ----------------------
    # 3. Validar duplicidad o redundancia
    # ----------------------
    if 'time_taken' in df.columns and 'time_in_minutes' in df.columns:
        df = df.drop(columns=['time_taken'])

    # ----------------------
    # 4. Validar y procesar 'is_outlier'
    # ----------------------
    if 'is_outlier' in df.columns:
        print(f"Outliers marcados en el dataset {dataset_name}: {df['is_outlier'].sum()}")

    # ----------------------
    # 5. Codificación de nuevas variables categóricas
    # ----------------------
    df = pd.get_dummies(df, columns=['day_of_week'], drop_first=True)
    for col in ['is_weekend', 'is_holiday', 'is_high_season']:
        df[col] = df[col].astype(int)

    # ----------------------
    # 6. Limpieza y ajustes finales
    # ----------------------
    # Convertir 'stop' a una métrica numérica
    if 'stop' in df.columns:
        df['stop_numeric'] = df['stop'].apply(
            lambda x: 0 if 'non-stop' in x else int(x.split()[0]) if isinstance(x, str) else np.nan
        )

    # Manejar valores nulos en las nuevas columnas
    df.fillna({'distance': 0, 'stop_numeric': 0}, inplace=True)

    # ----------------------
    # Retornar el DataFrame limpio y procesado
    # ----------------------
    return df
